## Video 5 : Building The Decision Tree Classification Model

### Importing the required libraries

In [1]:
#Importing libraries
import os
import pandas as pd
import numpy as np

### Displaying the first five rows

In [62]:
#Loading the data
os.chdir(r'C:\Users\Deepak\Downloads')
data = pd.read_csv('Synergix_data_preprocessed_new.csv')
data.head()


,Page_traffic,Unit_price,Units_sold,Segment,1_Star_Rating,2_Star_Rating,3_Star_Rating,4_Star_Rating,5_Star_Rating,Image_Count,...,Sum of Spend,Online_Clicks,Online_Cost,Online_Impressions,num_unique_campaigns,Clicks_video,Cost_video,Impressions_video,num_unique_campaigns_offline,Units_sold>1000
0,5835.0,22.214389,2071,Skincare,99.0,72.0,194.0,453.0,3272.0,6.0,...,0.0,1271.0,3102.448115,238075.0,6.0,3479.0,40244.28,3856872.0,2.0,1
1,1881.0,11.870778,681,Skincare,118.0,73.0,200.0,430.0,3084.0,9.0,...,0.0,494.0,1723.716359,75868.0,2.0,0.0,0.00,0.0,0.0,0
2,2477.0,27.645714,875,Makeup,108.0,85.0,232.0,511.0,3402.0,10.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0
3,4087.0,16.896848,1396,Makeup,82.0,64.0,167.0,303.0,2058.0,5.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,1
4,1446.0,16.357664,822,Hair Care,29.0,24.0,88.0,200.0,1221.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,6024.0,77040.81,7539723.0,2.0,0


<h3>Data preprocessing</h3>

In [3]:
#Looking for records with both 1 star and 2 star rating as 0
data[(data['1_Star_Rating'] == 0) & (data['2_Star_Rating'] == 0)]

,Page_traffic,Unit_price,Units_sold,Segment,1_Star_Rating,2_Star_Rating,3_Star_Rating,4_Star_Rating,5_Star_Rating,Image_Count,...,Sum of Spend,Online_Clicks,Online_Cost,Online_Impressions,num_unique_campaigns,Clicks_video,Cost_video,Impressions_video,num_unique_campaigns_offline,Units_sold>1000
6,4688.0,22.305753,1269,Makeup,0.0,0.0,0.0,0.0,0.0,7.0,...,837.408643,924.0,3462.254285,237797.0,6.0,0.0,0.00,0.0,0.0,1
9,4131.0,27.845638,1043,Skincare,0.0,0.0,25.0,66.0,403.0,9.0,...,0.000000,1168.0,4346.174220,260507.0,4.0,0.0,0.00,0.0,0.0,1
24,2048.0,12.632953,1226,Skincare,0.0,0.0,0.0,0.0,0.0,10.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,1
31,3124.0,23.365328,1021,Skincare,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,1216.0,4954.049764,153376.0,2.0,4330.0,56344.76,5344861.0,4.0,1
50,860.0,23.562358,441,Skincare,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19569,42.0,20.905660,0,Makeup,0.0,0.0,0.0,0.0,0.0,14.0,...,0.000000,0.0,0.000000,0.0,0.0,4330.0,56344.76,5344861.0,4.0,0
19575,2456.0,10.575479,1305,Skincare,0.0,0.0,27.0,61.0,402.0,6.0,...,385.387221,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,1
19590,3104.0,21.867328,1613,Makeup,0.0,0.0,0.0,0.0,0.0,8.0,...,0.000000,696.0,2308.875936,110614.0,2.0,11303.0,118307.34,9990498.0,4.0,1
19616,2371.0,6.938040,694,Makeup,0.0,0.0,0.0,0.0,0.0,9.0,...,236.060002,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0


In [4]:
#Storing the ratio in a list named Rating_ratio
Rating_ratio = []
for row in data.values:
    if(row[4]+row[5] == 0):
        if(row[7]+row[8] == 0):
            
            #If all the ratings are zero then overall rating ratio will also be zero
            Rating_ratio.append(0.0)
        
        else:
            #If only the denominator is zero then adding -99999 to the list temporarily
            Rating_ratio.append(-99999)
    
    #If sum of the bad ratings is non-zero, then we are calculating the ratio
    else:
        Rating_ratio.append((int(row[7])+(row[8]))/(int(row[4])+int(row[5])))

In [5]:
# Replacing -99999 with the maximum ratio in the list
max_rating = max(Rating_ratio)
for x in range(len(Rating_ratio)):
    if(Rating_ratio[x] == -99999):
        Rating_ratio[x] = max_rating

In [6]:
# Adding the column 'Good_By_Bad_Rating' to the dataframe
data['Good_By_Bad_Rating'] = Rating_ratio

In [7]:
data.head()

,Page_traffic,Unit_price,Units_sold,Segment,1_Star_Rating,2_Star_Rating,3_Star_Rating,4_Star_Rating,5_Star_Rating,Image_Count,...,Online_Clicks,Online_Cost,Online_Impressions,num_unique_campaigns,Clicks_video,Cost_video,Impressions_video,num_unique_campaigns_offline,Units_sold>1000,Good_By_Bad_Rating
0,5835.0,22.214389,2071,Skincare,99.0,72.0,194.0,453.0,3272.0,6.0,...,1271.0,3102.448115,238075.0,6.0,3479.0,40244.28,3856872.0,2.0,1,21.783626
1,1881.0,11.870778,681,Skincare,118.0,73.0,200.0,430.0,3084.0,9.0,...,494.0,1723.716359,75868.0,2.0,0.0,0.00,0.0,0.0,0,18.397906
2,2477.0,27.645714,875,Makeup,108.0,85.0,232.0,511.0,3402.0,10.0,...,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0,20.274611
3,4087.0,16.896848,1396,Makeup,82.0,64.0,167.0,303.0,2058.0,5.0,...,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,1,16.171233
4,1446.0,16.357664,822,Hair Care,29.0,24.0,88.0,200.0,1221.0,0.0,...,0.0,0.000000,0.0,0.0,6024.0,77040.81,7539723.0,2.0,0,26.811321


In [8]:
# Dropping the old columns related to ratings
data = data.drop(columns = ['1_Star_Rating', '2_Star_Rating', '3_Star_Rating', '4_Star_Rating', '5_Star_Rating'])

In [9]:
# Finding out the different categories in the column 'Segment'
unique_values = data ['Segment'].unique()
unique_values

array(['Skincare', 'Makeup', 'Hair Care'], dtype=object)

In [10]:
# Performing label encoding
from sklearn.preprocessing import LabelEncoder
data [['Segment']]= data [['Segment']].apply(LabelEncoder().fit_transform)

In [11]:
data.head()

,Page_traffic,Unit_price,Units_sold,Segment,Image_Count,Bullet_Count,Title_Count,Description_Length,num_unique_Search_Terms,Organic Search Rank,...,Online_Clicks,Online_Cost,Online_Impressions,num_unique_campaigns,Clicks_video,Cost_video,Impressions_video,num_unique_campaigns_offline,Units_sold>1000,Good_By_Bad_Rating
0,5835.0,22.214389,2071,2,6.0,6.0,89.0,141.0,3.0,3.0,...,1271.0,3102.448115,238075.0,6.0,3479.0,40244.28,3856872.0,2.0,1,21.783626
1,1881.0,11.870778,681,2,9.0,8.0,126.0,149.0,13.0,16.0,...,494.0,1723.716359,75868.0,2.0,0.0,0.00,0.0,0.0,0,18.397906
2,2477.0,27.645714,875,1,10.0,7.0,152.0,186.0,4.0,20.0,...,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0,20.274611
3,4087.0,16.896848,1396,1,5.0,7.0,117.0,209.0,7.0,20.0,...,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,1,16.171233
4,1446.0,16.357664,822,0,0.0,8.0,165.0,158.0,4.0,16.0,...,0.0,0.000000,0.0,0.0,6024.0,77040.81,7539723.0,2.0,0,26.811321


Here, the values of the segment have replaced with numerical values.

In [12]:
# Removing the continuous target variable from the dataframe
data = data.drop(columns = 'Units_sold',axis=1)

### Separating the feature variable and the target variable

In [13]:
# Separating the feature variable and the target variable
X = data.drop(columns = 'Units_sold>1000')
y = data['Units_sold>1000']

<h3>Performing train-test split</h3>

In [14]:
# Importing the train-test split from scikit-learn
from sklearn.model_selection import train_test_split

In [15]:
# Performing train and test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

<h3>Building the model</h3>

In [16]:
# Importing the DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier

In [17]:
# Building the decision tree classifier object
DT_model = DecisionTreeClassifier(random_state = 42)

In [18]:
# Building the model using the training data 
DT_model.fit(X_train, y_train)

DecisionTreeClassifier(random_state=42)

In [19]:
# Perfoming prediction on both the train and test data
y_pred_train = DT_model.predict(X_train)
y_pred = DT_model.predict(X_test)

<h3>Evaluating the model</h3>

In [20]:
# Importing the classification report from sklearn.metrics
from sklearn.metrics import classification_report

In [21]:
# Classification report on train data
class_report_test = classification_report(y_train,y_pred_train)
print ("The train report is:")
print (class_report_test)

The train report is:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5440
           1       1.00      1.00      1.00      8301

    accuracy                           1.00     13741
   macro avg       1.00      1.00      1.00     13741
weighted avg       1.00      1.00      1.00     13741



In [22]:
# Classification report on test data
class_report_test = classification_report(y_test,y_pred)
print ("The test report is:")
print (class_report_test)

The test report is:
              precision    recall  f1-score   support

           0       0.76      0.75      0.76      2354
           1       0.84      0.85      0.84      3536

    accuracy                           0.81      5890
   macro avg       0.80      0.80      0.80      5890
weighted avg       0.81      0.81      0.81      5890



<h3>Calculating feature importance</h3>

In [23]:
#Retreiving the importance score of each feature
feature_imp = DT_model.feature_importances_

In [24]:
# Sorting the features by importance in descending order in a dataframe.
imp = pd.DataFrame({'Col_names': X_train.columns, 'Importance': np.round(feature_imp, 2)})\
.sort_values(by='Importance', ascending=False)
imp

,Col_names,Importance
0,Page_traffic,0.32
1,Unit_price,0.16
19,Good_By_Bad_Rating,0.10
5,Title_Count,0.07
3,Image_Count,0.05
6,Description_Length,0.05
8,Organic Search Rank,0.05
7,num_unique_Search_Terms,0.04
12,Online_Cost,0.03
13,Online_Impressions,0.03


In [25]:
# Calculating the cumulative sum of the 'Importance' column and store it in a new column called 'cum_imp'
imp['cum_imp'] = imp.Importance.cumsum()
imp

,Col_names,Importance,cum_imp
0,Page_traffic,0.32,0.32
1,Unit_price,0.16,0.48
19,Good_By_Bad_Rating,0.10,0.58
5,Title_Count,0.07,0.65
3,Image_Count,0.05,0.70
6,Description_Length,0.05,0.75
8,Organic Search Rank,0.05,0.80
7,num_unique_Search_Terms,0.04,0.84
12,Online_Cost,0.03,0.87
13,Online_Impressions,0.03,0.90


In [26]:
# Selecting the columns with cumulative importance exceeding 0.90 and create a list of column names to drop
drop_col = imp[imp.cum_imp >  0.90]['Col_names'].to_list()
drop_col

['Online_Impressions',
 'Online_Clicks',
 'num_unique_campaigns',
 'Bullet_Count',
 'Segment',
 'Clicks_video',
 'Cost_video',
 'Impressions_video',
 'Sum of Spend',
 'Num_of_Promotions',
 'num_unique_campaigns_offline']

In [27]:
# Dropping the columns from X_train
X_train.drop(columns = drop_col, axis = 1, inplace = True)

# Dropping the columns from X_test
X_test.drop(columns = drop_col, axis = 1, inplace = True)

<h3>Making predictions using the model</h3>

In [28]:
# Building the model again using the modified X_train
DT_model = DecisionTreeClassifier(random_state = 42)

In [29]:
DT_model.fit(X_train, y_train)

DecisionTreeClassifier(random_state=42)

In [30]:
# Making the predictions using the modified X_test
y_pred = DT_model.predict(X_test)

<h3>Evaluating the model</h3>

In [31]:
#Classification report on train data
class_report_test = classification_report(y_train,y_pred_train)
print ("The train report is:")
print (class_report_test)

The train report is:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5440
           1       1.00      1.00      1.00      8301

    accuracy                           1.00     13741
   macro avg       1.00      1.00      1.00     13741
weighted avg       1.00      1.00      1.00     13741



In [32]:
# Classification report on test data
class_report_test = classification_report(y_test,y_pred)
print ("The test report is:")
print (class_report_test)

The test report is:
              precision    recall  f1-score   support

           0       0.75      0.74      0.75      2354
           1       0.83      0.84      0.83      3536

    accuracy                           0.80      5890
   macro avg       0.79      0.79      0.79      5890
weighted avg       0.80      0.80      0.80      5890



---

<h2> Video 7: Hyperparameter Tuning for Model Optimization </h2>



In [33]:
# Storing the depth of our current model
depth = DT_model.get_depth()

In [34]:
depth

26

In [35]:
# List of max_depth values 
max_depth_list = list(range(depth,0,-3))

In [36]:
# Importing the F1_score from sklearn.metrics
from sklearn.metrics import f1_score

In [213]:
# Dictionary to store the train and test F1 scores

train_scores = {}
test_scores = {}

# Loop through max_depth values and train the models
for depth in max_depth_list:
    # Initialize the Decision Tree model with the current max_depth value
    DT_model = DecisionTreeClassifier(max_depth=depth, random_state=42)
    
    # Train the model
    DT_model.fit(X_train, y_train)
    
    # Make predictions on the train dataset
    y_train_pred = DT_model.predict(X_train)

    # Make predictions on the test dataset
    y_test_pred = DT_model.predict(X_test)
    
    # Store the train F1 score in the dictionary with the max_depth as the key
    train_scores[depth] = f1_score(y_train, y_train_pred)
    
    # Store the test F1 score in the dictionary with the max_depth as the key
    test_scores[depth] = f1_score(y_test, y_test_pred)
    

# Print the train and test F1 scores for each model
for depth in max_depth_list:
    print(f"max_depth = {depth}|\
    Train Score = {train_scores[depth]:.3f} |\
    Test score = {test_scores[depth]:.3f}")
    print('_'*65)

max_depth = 26|    Train Score = 1.000 |    Test score = 0.841
_________________________________________________________________
max_depth = 23|    Train Score = 0.999 |    Test score = 0.836
_________________________________________________________________
max_depth = 20|    Train Score = 0.995 |    Test score = 0.839
_________________________________________________________________
max_depth = 17|    Train Score = 0.981 |    Test score = 0.839
_________________________________________________________________
max_depth = 14|    Train Score = 0.949 |    Test score = 0.850
_________________________________________________________________
max_depth = 11|    Train Score = 0.905 |    Test score = 0.849
_________________________________________________________________
max_depth = 8|    Train Score = 0.853 |    Test score = 0.834
_________________________________________________________________
max_depth = 5|    Train Score = 0.809 |    Test score = 0.810
____________________________________

In [214]:
# Dictionary to store the train and test F1 scores

train_scores = {}
test_scores = {}


# Fill the values in the below list and assign value to the variable!
min_sample_leaf_list = 
depth = 

# Loop through min_sample_leaf values and train the models
for min_sample_leaf in min_sample_leaf_list:
    
    # Initialize the Decision Tree model with the current min_samples_leaf value
    DT_model = DecisionTreeClassifier(min_samples_leaf=min_sample_leaf, max_depth= depth, random_state=42)
    
    # Train the model
    DT_model.fit(X_train, y_train)
    
    # Make predictions on the train dataset
    y_train_pred = DT_model.predict(X_train)

    # Make predictions on the test dataset
    y_test_pred = DT_model.predict(X_test)
    
    # Store the train F1 score in the dictionary with the min_sample_leaf as the key
    train_scores[min_sample_leaf] = f1_score(y_train, y_train_pred)
    
    # Store the test F1 score in the dictionary with the min_sample_leaf as the key
    test_scores[min_sample_leaf] = f1_score (y_test, y_test_pred)
    

# Print the train and test F1 scores for each model
for min_sample_leaf in min_sample_leaf_list:
    print(f"min_sample_leaf = {min_sample_leaf}|\
    Train Score = {train_scores[min_sample_leaf]:.3f} |\
    Test Score = {test_scores[min_sample_leaf]:.3f}")
    print('_'*65)

min_sample_leaf = 10|    Train Score = 0.847 |    Test Score = 0.833
_________________________________________________________________
min_sample_leaf = 9|    Train Score = 0.847 |    Test Score = 0.833
_________________________________________________________________
min_sample_leaf = 8|    Train Score = 0.848 |    Test Score = 0.832
_________________________________________________________________
min_sample_leaf = 7|    Train Score = 0.848 |    Test Score = 0.832
_________________________________________________________________
min_sample_leaf = 6|    Train Score = 0.848 |    Test Score = 0.833
_________________________________________________________________
min_sample_leaf = 5|    Train Score = 0.849 |    Test Score = 0.833
_________________________________________________________________
min_sample_leaf = 4|    Train Score = 0.850 |    Test Score = 0.832
_________________________________________________________________
min_sample_leaf = 3|    Train Score = 0.851 |    Test Score =